# Pandas字符串操作

前面我们已经使用了字符串的处理函数：<br>
df["气温(度)"].str.replace("℃", "").astype("float")

***Pandas的字符串处理:***
1. 使用方法: 先获取Series的str属性，然后再属性上调用函数;
2. **只能在字符串列上使用**，不能再数字列上使用;
3. DataFrame上没有str属性和处理方法;
4. Series.str并不是Python原生字符串，而是自己的一套方法，不过大部分和原生str很相似;

***Series.str字符串方法列表参考文档:***<br>
https://pandas.pydata.org/docs/reference/series.html

***本节演示内容:***
1. 获取Series的str属性，然后使用各种字符串处理函数
2. 使用str的startswith、contains等boolean类Series可以做条件查询
3. 需要多次str处理的链式操作
4. 使用正则表达式处理

## 0. 读取天气数据

In [1]:
import pandas as pd

In [2]:
## 0. 读取数据
fpath = "../datas/weather_20230115134249.csv"
df = pd.read_csv(fpath)

In [3]:
df.head()

,日期,城市,行政区,观测站,气温(度),相对湿度(%),累积雨量(mm)
0,2015-01-01,新北市,烏來區,福山,13.7℃,92,0.0
1,2015-01-02,臺南市,安平區,安平,23.5℃,70,0.0
2,2015-01-03,臺東縣,東河鄉,七塊厝,19.6℃,86,0.0
3,2015-01-04,新北市,貢寮區,福隆,14.2℃,96,-99.0
4,2015-01-05,南投縣,仁愛鄉,小奇萊,8.3℃,57,0.0


In [4]:
df.dtypes

日期           object
城市           object
行政区          object
观测站          object
气温(度)        object
相对湿度(%)       int64
累积雨量(mm)    float64
dtype: object

## 1. 获取Series的str属性，使用各种字符串处理函数

In [12]:
df["气温(度)"].str

In [13]:
df["气温(度)"].str.replace("℃", "").head()

0    13.7
1    23.5
2    19.6
3    14.2
4     8.3
Name: 气温(度), dtype: object

In [14]:
# 判断是不是数字
df["气温(度)"].str.isnumeric().head()

0    False
1    False
2    False
3    False
4    False
Name: 气温(度), dtype: bool

In [15]:
df["相对湿度(%)"].str.len()

AttributeError: Can only use .str accessor with string values!

## 2. 使用str的startswith、contains等得到bool的Series可以做条件查询

In [19]:
condition = df["日期"].str.startswith("2015-02")

In [20]:
condition.head()

0    False
1    False
2    False
3    False
4    False
Name: 日期, dtype: bool

In [21]:
df[condition].head()

,日期,城市,行政区,观测站,气温(度),相对湿度(%),累积雨量(mm)
31,2015-02-01,臺南市,新化區,虎頭埤,26.1℃,67,0.0
32,2015-02-02,臺中市,石岡區,石岡,19.4℃,99,0.0
33,2015-02-03,屏東縣,枋山鄉,枋山,25.1℃,67,0.0
34,2015-02-04,屏東縣,滿州鄉,滿州,22.6℃,79,0.0
35,2015-02-05,屏東縣,恆春鎮,墾雷,23.6℃,82,0.0


## 3. 需要多次str处理的链式操作

怎么提取201811这样的数字月份？<br>
1.现将日期2018-11-20替换成20181120的形式<br>
2.提取月份字符串201811

In [22]:
df["日期"].str.replace("-", "").head()

0    20150101
1    20150102
2    20150103
3    20150104
4    20150105
Name: 日期, dtype: object

In [23]:
# 每次调用函数，都返回一个新的Series
df["日期"].str.replace("-", "").slice(0, 6).head()

AttributeError: 'Series' object has no attribute 'slice'

In [24]:
df["日期"].str.replace("-", "").str.slice(0, 6).head()

0    201501
1    201501
2    201501
3    201501
4    201501
Name: 日期, dtype: object

In [25]:
# slice就是切片语法，可以直接用
df["日期"].str.replace("-", "").str[0:6].head()

0    201501
1    201501
2    201501
3    201501
4    201501
Name: 日期, dtype: object

## 4. 使用正则表达式的处理

In [38]:
# 添加新列
def get_date(x):
    year, month, day = x["日期"].split("-")
    return f"{year}年{month}月{day}日"


df["中文日期"] = df.apply(get_date, axis=1)

In [39]:
df["中文日期"].head()
df

,日期,城市,行政区,观测站,气温(度),相对湿度(%),累积雨量(mm),中文日期
0,2015-01-01,新北市,烏來區,福山,13.7℃,92,0.0,2015年01月01日
1,2015-01-02,臺南市,安平區,安平,23.5℃,70,0.0,2015年01月02日
2,2015-01-03,臺東縣,東河鄉,七塊厝,19.6℃,86,0.0,2015年01月03日
3,2015-01-04,新北市,貢寮區,福隆,14.2℃,96,-99.0,2015年01月04日
4,2015-01-05,南投縣,仁愛鄉,小奇萊,8.3℃,57,0.0,2015年01月05日
...,...,...,...,...,...,...,...,...
479,2016-04-24,新北市,板橋區,板橋,15.6℃,73,5.0,2016年04月24日
480,2016-04-25,宜蘭縣,大同鄉,翠峰湖,7.8℃,100,0.0,2016年04月25日
481,2016-04-26,花蓮縣,壽豐鄉,大坑,15.7℃,94,0.0,2016年04月26日
482,2016-04-27,新竹縣,五峰鄉,雪霸,15.7℃,62,0.0,2016年04月27日


问题：怎么将"2015年11月20日"中的年、月、日三个中文字符去除？

In [33]:
# 方法1：链式replace
df["中文日期"].str.replace("年", "").str.replace("月", "").str.replace("日", "").head()

0    20150101
1    20150102
2    20150103
3    20150104
4    20150105
Name: 中文日期, dtype: object

***Series.str默认就开启了正则表达式模式***

In [53]:
# 方法二：正则表达式替换
df["中文日期"].str.replace("[年|月|日]", "", regex=True).head()

0    20150101
1    20150102
2    20150103
3    20150104
4    20150105
Name: 中文日期, dtype: object